## Glycemic control in critically patients with and without diabetes

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
import psycopg2 as pg
connection = pg.connect(database="eicu", user="postgres", password="******", host="127.0.0.1", port="5432") 

In [ ]:
# 1. patient
query = """
select *
from patient
"""
df_patient = pd.read_sql_query(query, con=connection)

#2. admissiondrug
query = """
select 
  admissiondrugid, patientunitstayid,
  drugenteredoffset
  , drugname
from admissiondrug
"""
df_admissiondrug = pd.read_sql_query(query, con=connection)

#3. pasthistory
query = """
select 
  patientunitstayid, pasthistorypath
from pasthistory
"""
df_pasthistory = pd.read_sql_query(query, con=connection)

query = """
select patientunitstayid, activeupondischarge, diagnosisstring
from diagnosis
"""
df_diagnosis = pd.read_sql_query(query, con=connection)

# 4. admissiondx
# To decide whether the patients' admission diagnosis was operative or non-operative
# By looking for the answer of the statement:
# "admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|No"
# or "admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|Yes"
query = """
select *
from admissiondx
WHERE admitdxpath LIKE '%O.R.%' 
"""
df_admissiondx_operative = pd.read_sql_query(query, con=connection)

query = """
select *
from admissiondx
WHERE admitdxpath LIKE '%Elective%' 
"""
df_admissiondx_elective = pd.read_sql_query(query, con=connection)

# 5. apacheApsVar, apachePredVar
# apacheApsVar contains the variables used to calculate the Acute Physiology Score (APS) III for patients.
query = """
select *
from apacheApsVar
"""
df_apacheApsVar = pd.read_sql_query(query, con=connection)

query = """
select *
from apachePredVar
"""
df_apachePredVar = pd.read_sql_query(query, con=connection)

# 6. apachePatientResult 
# Select only results from APAHCE IV instead of IVa
# Including hospital and ICU mortality, hospital and ICU length of stays
query = """
select *
from apachePatientResult
where apacheversion='IV';
"""
df_apachePatientResult = pd.read_sql_query(query, con=connection)

# 7. lab & customlab
# customlab contained some extra results for infrequent, less standardised laboratory tests
query = """
select patientunitstayid, labname, labresultoffset, labresult, labmeasurenamesystem
from lab
"""
df_lab = pd.read_sql_query(query, con=connection)
query = """
select patientunitstayid, labotheroffset, labothername, labotherresult, labothervaluetext
from customlab
"""
df_customlab = pd.read_sql_query(query, con=connection)

# 8. treatment 
query = """
select *
from treatment
"""
df_treatment = pd.read_sql_query(query, con=connection)

# 9. medication
query = """
select patientunitstayid, drugname, drugorderoffset
from medication
"""
df_medication = pd.read_sql_query(query, con=connection)

# 10.infusionDrug
query = """
select patientunitstayid, infusionoffset, drugname, drugrate
from infusiondrug
"""
df_infusionDrug = pd.read_sql_query(query, con = connection)

In [942]:
df_patient["age"]=df_patient["age"].replace(to_replace=['> 89',''], value=[90,np.NaN])
df_patient["age"]=df_patient["age"].astype(float)

In [943]:
df_patient["hospitaldischargestatus_patient"]=df_patient["hospitaldischargestatus"].replace(to_replace=['Alive', 'Expired',''], value=[0, 1, -1])
df_patient["unitdischargestatus_patient"]=df_patient["unitdischargestatus"].replace(to_replace=['Alive', 'Expired',''], value=[0, 1, -1])
# Since this table was not the only source of information for discharge status, 
# we would keep the column hospitaldischargeststus and unitdischargestatus meanwhile
# and the duplicate column would be dropped later

In [ ]:
# pick the first ICU stay for each unique patient
# Reference: http://github.com/MIT-LCP/eicu-code/issues/16
df_uniquepatient_raw=df_patient.sort_values(["uniquepid", "patienthealthsystemstayid", "unitvisitnumber"]).groupby(["uniquepid"]).first()
df_uniquepatient_raw.reset_index(inplace=True)
print(df_uniquepatient_raw.shape) #corresponding to the number of unique patients

#Removing unnecessary variables
df_uniquepatient=df_uniquepatient_raw.drop(columns=["patienthealthsystemstayid" ,"uniquepid", "wardid",  "hospitaladmittime24", "unitadmittime24", "hospitaldischargeyear", "hospitaldischargetime24", "unitvisitnumber", "unitstaytype", "unitdischargetime24", "unitdischargelocation", "hospitaldischargelocation"])

In [951]:
#Removing repeat ICU stays
df_uniquepatient=df_uniquepatient[df_uniquepatient['unitadmitsource']!="Other ICU"]
df_uniquepatient=df_uniquepatient[df_uniquepatient['unitadmitsource']!="ICU"]
df_uniquepatient=df_uniquepatient[df_uniquepatient['unitadmitsource']!="ICU to SDU"]

Both bedside glucose measurement and laboratory glucose measurement would be selected.

In [ ]:
#Both bedside glucose measurement and laboratory glucose measurement would be selected.
df_lab_glucose=df_lab[(df_lab['labname']=='bedside glucose')|(df_lab['labname']=='glucose')]

#select results obtained during ICU stay only
df_lab_glucose=df_lab_glucose[df_lab_glucose.labresultoffset>0] 

In [958]:
#Mortality
df_uniquepatient=df_uniquepatient.merge(df_apachePatientResult[["patientunitstayid", "actualhospitalmortality", "actualicumortality"]], on="patientunitstayid", how="left")
df_uniquepatient["actualhospitalmortality"]=df_uniquepatient["actualhospitalmortality"].replace(to_replace=['ALIVE', 'EXPIRED',''], value=[0, 1,-1])
df_uniquepatient["actualicumortality"]=df_uniquepatient["actualicumortality"].replace(to_replace=['ALIVE', 'EXPIRED',''], value=[0, 1,-1])

# Extract hospital and icu discharge status from patient.csv and apachepatientresult.csv
df_uniquepatient["hospitaldischargestatus"]=df_uniquepatient[["hospitaldischargestatus_patient","actualhospitalmortality"]].max(axis=1)
df_uniquepatient["unitdischargestatus"]=df_uniquepatient[["unitdischargestatus_patient","actualicumortality"]].max(axis=1)
#convert -1 back to nan
df_uniquepatient["hospitaldischargestatus"]=df_uniquepatient["hospitaldischargestatus"].replace(to_replace=[0,1,-1], value=[0, 1,np.nan])
df_uniquepatient["unitdischargestatus"]=df_uniquepatient["unitdischargestatus"].replace(to_replace=[0,1,-1], value=[0, 1,np.nan])

# Remove duplicated columns
df_uniquepatient=df_uniquepatient.drop(columns=["hospitaldischargestatus_patient", "unitdischargestatus_patient", "actualhospitalmortality", "actualicumortality"])

In [ ]:
#Excluding patients with no hospital mortality data (primary outcome)
df_uniquepatient=df_uniquepatient[df_uniquepatient["hospitaldischargestatus"].notnull()]

In [961]:
#LOS
df_uniquepatient["icu_los"]=df_uniquepatient["unitdischargeoffset"]/1440 #convert to days
df_uniquepatient["hospital_los"]=(df_uniquepatient["hospitaldischargeoffset"]-df_uniquepatient["hospitaladmitoffset"])/1440
df_uniquepatient=df_uniquepatient.drop(columns=["hospitaladmitoffset", "hospitaldischargeoffset", "unitdischargeoffset"])

Extracting the diagnosis of diabetes from tables
* diagnosis
* pasthistory
* apachePredVar
* admissiondrug

HbA1c was not available from table *lab* and *customlab*

In [ ]:
#1. diagnosis
df_diagnosis=df_diagnosis[df_diagnosis["activeupondischarge"]==True]
df_diagnosis["diabetes"]=df_diagnosis['diagnosisstring'].str.contains('diabetes mellitus') #limited to patient with diabetes

#Create a record for the presence or absence of DM in each patientunitstayid
df_diagnosis_DM=df_diagnosis.groupby("patientunitstayid")["diabetes"].sum()
#convert to dataframe
df_diagnosis_DM=pd.DataFrame(df_diagnosis_DM)
df_diagnosis_DM.rename(columns={"diabetes":"DMdiagnosis"}, inplace=True)
df_diagnosis_DM

In [ ]:
# 2. pasthistory
df_pasthistory_DM=pd.DataFrame()
df_pasthistory["IDDM"]=df_pasthistory['pasthistorypath'].str.contains('insulin dependent diabetes')
df_pasthistory_DM["IDDM"]=df_pasthistory.groupby(["patientunitstayid"])["IDDM"].sum()>0

df_pasthistory["NIDDM_OHA"]=df_pasthistory['pasthistorypath'].str.contains('Non-Insulin Dependent Diabetes/medication dependent')
df_pasthistory_DM["NIDDM_OHA"]=df_pasthistory.groupby(["patientunitstayid"])["NIDDM_OHA"].sum()>0

df_pasthistory["NIDDM_diet"]=df_pasthistory['pasthistorypath'].str.contains('Non-Insulin Dependent Diabetes/non-medication dependent')
df_pasthistory_DM["NIDDM_diet"]=df_pasthistory.groupby(["patientunitstayid"])["NIDDM_diet"].sum()>0

In [972]:
# 3. apachePredVar
df_apachePredVar["DM_apache"]=df_apachePredVar["diabetes"]

In [974]:
# 4. admissiondrug
#Based on the result of manual review of all the unique medications
insulin_name= ["NOVOLOG", "LANTUS", "INSULIN", "HUMULIN", "NOVOLIN", "LEVEMIR", "HUMALOG","DETEMIR"]
OHA_name=["GLYBURIDE", "GLUCOPHAGE", "JANUMET", "GLIMEPIRIDE", "GLIPIZIDE", "GLUCOTROL", "METFORMIN", "GLYBURIDE/METFORMIN", \
          "AMARYL", "SITAGLIPTIN PHOSPHATE", "JANUVIA", "GLUCOTROL XL", "ACTOS", "GLUCOVANCE", "SAXAGLIPTIN HCL", "PIOGLITAZONE HCL",\
          "GLYBURIDE-METFORMIN", "CANAGLIFLOZIN", "LINAGLIPTIN", "ACTOPLUS MET", "GLUMETZA", "REPAGLINIDE", "DIABETA", "JARDIANCE",\
          "PIOGLITAZONE HCL/METFORMIN HCL", "INVOKAMET", "RIOMET"]
insulin = '|'.join(insulin_name)
OHA = '|'.join(OHA_name)
df_admissiondrug["insulin"]=df_admissiondrug['drugname'].str.contains(insulin) 
df_admissiondrug["OHA"]=df_admissiondrug['drugname'].str.contains(OHA) 
# GLP1 agonist VICTOZA 2-PAK / LIRAGLUTIDE was not included because 1) it is usually used with other diabetic medications/ insulin, 2) its indication also extends to weight management

df_admission_DMmeds=pd.DataFrame()
df_admission_DMmeds["insulin"]= df_admissiondrug.groupby(["patientunitstayid"])["insulin"].sum()>0
df_admission_DMmeds["OHA"]= df_admissiondrug.groupby(["patientunitstayid"])["OHA"].sum()>0
df_admission_DMmeds

In [977]:
df_uniquepatient=df_uniquepatient.merge(df_diagnosis_DM, on='patientunitstayid', how='left')
df_uniquepatient=df_uniquepatient.merge(df_pasthistory_DM, on='patientunitstayid', how='left')
df_uniquepatient=df_uniquepatient.merge(df_admission_DMmeds, on='patientunitstayid', how='left')
df_uniquepatient=df_uniquepatient.merge(df_apachePredVar[["DM_apache", "patientunitstayid", "ventday1"]], on='patientunitstayid', how='left')

# A patient is defined to have diabetes based on 
# the table diagnosis, the table pasthistory, and the table admissiondrug
df_uniquepatient["diabetes"]=(df_uniquepatient["DMdiagnosis"]>0) | \
(df_uniquepatient["DM_apache"]>0) |\
(df_uniquepatient["IDDM"]) | \
(df_uniquepatient["NIDDM_OHA"]) | \
(df_uniquepatient["NIDDM_diet"]) | \
(df_uniquepatient["insulin"]) |  \
(df_uniquepatient["OHA"])

In [ ]:
# Numbers add up exist the number of patients with diabetes because of overlapping labelling
# *1 Patients who were on insulin -> IDDM
# *2 Patients who were on OHA but not insulin -> NIDDM on OHA
# *3 Patients who were on diet but not insulin or OHA -> NIDDM on diet

#Remove overlapping labels
# 1. Changed all labelled NIDDM to False for whose who were on insulin
df_uniquepatient.loc[df_uniquepatient["insulin"]==True, "NIDDM_OHA"]=False
df_uniquepatient.loc[df_uniquepatient["insulin"]==True, "NIDDM_diet"]=False

# 2. Changed all overlapping NIDDM and IDDM to IDDM 
df_uniquepatient.loc[df_uniquepatient["IDDM"]==True, "NIDDM_OHA"]=False
df_uniquepatient.loc[df_uniquepatient["IDDM"]==True, "NIDDM_diet"]=False

# 3. Remove duplicate label of NIDDM_diet in patients already taking OHA
df_uniquepatient.loc[df_uniquepatient["NIDDM_OHA"]==True, "NIDDM_diet"]=False

#Recheck the diagnosis of DM
df_uniquepatient["diabetes"]=(df_uniquepatient["DMdiagnosis"]>0) | \
(df_uniquepatient["DM_apache"]>0) |\
(df_uniquepatient["IDDM"]) | \
(df_uniquepatient["NIDDM_OHA"]) | \
(df_uniquepatient["NIDDM_diet"]) | \
(df_uniquepatient["insulin"]) |  \
(df_uniquepatient["OHA"])

# Drop unnecessary columns
df_uniquepatient=df_uniquepatient.drop(columns=["OHA", "insulin", "DMdiagnosis", "DM_apache"])

In [ ]:
#Label diabetic patients who were neither labelled IDDM/ NIDDM as DM_unknownstatus
df_uniquepatient.loc[df_uniquepatient["diabetes"] == 1, "DM_status"] = "DM"
df_uniquepatient.loc[df_uniquepatient["diabetes"] == 0, "DM_status"] = "No_DM"
df_uniquepatient.loc[df_uniquepatient["IDDM"] == 1, "DM_status"] = "IDDM"
df_uniquepatient.loc[df_uniquepatient["NIDDM_OHA"] == 1, "DM_status"] = "NIDDM_OHA"
df_uniquepatient.loc[df_uniquepatient["NIDDM_diet"] == 1, "DM_status"] = "NIDDM_diet"
df_uniquepatient.loc[df_uniquepatient["DM_status"] == "DM", "DM_status"] = "DM_unknownstatus"

In [985]:
#Decide whether patients were admitted for medical or surgical diagnosis
df_admissiondx_operative["operative"]=df_admissiondx_operative["admitdxpath"].replace \
(to_replace=['admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|No', \
             'admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|Yes'], \
 value=['0', '1']).astype(float)
df_uniquepatient=df_uniquepatient.merge(df_admissiondx_operative[["patientunitstayid", "operative"]], on='patientunitstayid', how='left')

In [ ]:
#Decide whether patients were admitted for elective operation or emergency operation
df_admissiondx_elective["elective"]=df_admissiondx_elective["admitdxpath"].replace \
(to_replace=['admission diagnosis|Elective|No', \
             'admission diagnosis|Elective|Yes'], \
 value=['0', '1']).astype(float)
df_uniquepatient=df_uniquepatient.merge(df_admissiondx_elective[["patientunitstayid", "elective"]], on='patientunitstayid', how='left')

In [991]:
#Include APACHE IV scores
df_uniquepatient=df_uniquepatient.merge(df_apachePatientResult[["patientunitstayid", "apachescore"]], on="patientunitstayid", how="left")

In [993]:
#Determine whether patients who put on inotropes/ vasopressors from tables
#1) infusionDrug
inotropevasopressor_name=['milrinone', 'norepinephrine', 'vasopressin', 'epinephrine', 'dopamine', 'dobutamine', 'dobutrex', 'levophed']
inotropevasopressor = '|'.join(inotropevasopressor_name)
df_infusionDrug["inotropevasopressor"]=df_infusionDrug["drugname"].str.lower().str.contains(inotropevasopressor)

#2) treatment
inotropevasopressor_name=['inotropic agent', 'vasopressors']
inotropevasopressor= '|'.join(inotropevasopressor_name)
df_treatment["treatment_inotropevasopressor"]=df_treatment['treatmentstring'].str.contains(inotropevasopressor) 

df_treatment_filtered["treatment_inotropevasopressor"]=df_treatment.groupby(["patientunitstayid"])["treatment_inotropevasopressor"].sum()>0
df_treatment_filtered=pd.DataFrame(df_treatment_filtered)

# Merging data into df_uniquepatient
df_uniquepatient=df_uniquepatient.merge(df_treatment_filtered, on='patientunitstayid', how='left')
df_uniquepatient=df_uniquepatient.merge(df_infusionDrug_filtered_inotropevasopressor, on='patientunitstayid', how='left')
# extracting data fro minfusiondrug and treatment table
df_uniquepatient["inotropevasopressor"]=(df_uniquepatient["infusion_inotropevasopressor"]) | (df_uniquepatient["treatment_inotropevasopressor"])
# Drop unnecessary columns
df_uniquepatient.drop(columns=["infusion_inotropevasopressor", "treatment_inotropevasopressor"], inplace=True)

In [ ]:
#Labelling trauma patients
df_uniquepatient["trauma"]=df_uniquepatient["apacheadmissiondx"].str.lower().str.contains('trauma', na=False) 

#Exclude non-trauma
df_uniquepatient["trauma"][df_uniquepatient['apacheadmissiondx'].str.contains("see") == True]= False
#Example: Fracture-pathological, non-union, non-traumatic, for fractures due to trauma see Trauma
df_uniquepatient["trauma"][df_uniquepatient['apacheadmissiondx'].str.contains("non-traumatic") == True]= False
#Examples: Chest/multiple trauma', 'Amputation (non-traumatic)

In [ ]:
#Labelling DKA and HONK
df_uniquepatient["DKA"]=df_uniquepatient["apacheadmissiondx"].str.lower().str.contains('diabetic ketoacidosis', na=False) 
df_uniquepatient["HONK"]=df_uniquepatient["apacheadmissiondx"].str.lower().str.contains('diabetic hyperglycemic hyperosmolar', na=False) 

In [ ]:
#Calculate the mean, sd, cv, and find the maximum and minimum glucose level
#the mean glucose from bedside glucose measurement and laboratory glucose measurement was also calculated
df_glucose=pd.DataFrame()
df_lab_glucose.dropna(inplace=True)
df_glucose["glucose_mean"] = df_lab_glucose.groupby(["patientunitstayid"])["labresult"].mean()
df_glucose["glucose_sd"] = df_lab_glucose.groupby(["patientunitstayid"])["labresult"].std()
df_glucose["glucose_cv"] = df_lab_glucose.groupby(["patientunitstayid"])["labresult"].std()/ df_lab_glucose.groupby(["patientunitstayid"])["labresult"].mean()*100
df_glucose["glucose_max"] = df_lab_glucose.groupby(["patientunitstayid"])["labresult"].max()
df_glucose["glucose_min"] = df_lab_glucose.groupby(["patientunitstayid"])["labresult"].min()
df_glucose["count"] = df_lab_glucose.groupby(["patientunitstayid"])["labresult"].count()
df_glucose["bedsideglucose_mean"] = df_lab_glucose[df_lab_glucose['labname']=='bedside glucose'].groupby(["patientunitstayid"])["labresult"].mean()
df_glucose["labglucose_mean"] = df_lab_glucose[df_lab_glucose['labname']=='glucose'].groupby(["patientunitstayid"])["labresult"].mean()

# merge with df_uniquepatient
df_uniquepatient=df_uniquepatient.merge(df_glucose, on="patientunitstayid", how="left")

In [ ]:
#Calculate the time-weighted average mean glucose for each patientunitid
grouped = df_lab_glucose.sort_values(['patientunitstayid','labresultoffset']).groupby("patientunitstayid", as_index = False)
grouped_index = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index()
grouped_index.set_index(['level_0', 'level_1'], inplace=True)

df_twamean=pd.DataFrame()
patientid=[]
twamean=[]
for i in grouped_index.index.levels[0]: #each patientunitstayid
    mean=0
    time_diff=0
    twa=0
    auc=0
    min_idx=grouped_index.loc[i].index.min()
    max_idx=grouped_index.loc[i].index.max()
    for j in range(max_idx): 
        #each result within one patientunitstayid sorted by labresultoffset
            mean=(grouped_index.loc[i,j]["labresult"]+grouped_index.loc[i,j+1]["labresult"])
            time_diff=grouped_index.loc[i,j+1]["labresultoffset"]-grouped_index.loc[i,j]["labresultoffset"]
            # labresultoffset in munutes
            auc+=(mean*time_diff)
    auc=auc/2
    twa=auc/(grouped_index.loc[i,max_idx]["labresultoffset"]-grouped_index.loc[i,min_idx]["labresultoffset"])
    patientid.append(grouped_index.loc[i,0]["patientunitstayid"])
    twamean.append(twa)
patientid=pd.Series(patientid, name='patientunitstayid')
twamean=pd.Series(twamean, name='glucose_twamean')        
df_twamean=pd.concat([patientid, twamean], axis=1)
df_uniquepatient=df_uniquepatient.merge(df_twamean, on="patientunitstayid", how="left")       

In [ ]:
#Exclude patients with less than 2 glucose measurements
df_uniquepatient=df_uniquepatient.dropna(subset=['glucose_twamean'])

In [1037]:
#Calculate and categorise BMI
df_uniquepatient['BMI']=df_uniquepatient["admissionweight"]/ ((df_patient["admissionheight"]/100)**2)
df_uniquepatient.loc[df_uniquepatient["BMI"] <18.5, "BMI_cat"] = "<18.5"
df_uniquepatient.loc[(df_uniquepatient["BMI"] >=18.5) & (df_uniquepatient["BMI"] <25), "BMI_cat"] = "18.5-25"
df_uniquepatient.loc[(df_uniquepatient["BMI"] >=25) & (df_uniquepatient["BMI"] <30), "BMI_cat"] = "25-<30"
df_uniquepatient.loc[(df_uniquepatient["BMI"] >=30) & (df_uniquepatient["BMI"] <35), "BMI_cat"] = "30-<35"
df_uniquepatient.loc[(df_uniquepatient["BMI"] >=35) & (df_uniquepatient["BMI"] <40), "BMI_cat"] = "35-<40"
df_uniquepatient.loc[df_uniquepatient["BMI"] >=40, "BMI_cat"] = ">=40"

In [ ]:
#Exclude DKA and HONK patients
df_uniquepatient=df_uniquepatient[df_uniquepatient["DKA"]==0]
df_uniquepatient=df_uniquepatient[df_uniquepatient["HONK"]==0]

In [ ]:
#Exclude ICU LOS<2 days
df_uniquepatient_morethan2=df_uniquepatient[df_uniquepatient["icu_los"]>=2]

In [ ]:
#Preparing data for statistical modelling in R
from sklearn.model_selection import train_test_split
X_train, X_test, _, _ = train_test_split(df_uniquepatient_morethan2_nonan, df_uniquepatient_morethan2_nonan["hospitaldischargestatus"], test_size=0.3, random_state=42, stratify=df_uniquepatient_morethan2_nonan["hospitaldischargestatus"])
X_train.to_csv("X_train.csv")
X_train_DM=X_train[X_train["diabetes"]==1]
X_train_DM.to_csv("X_train_DM.csv")
X_train_nonDM=X_train[X_train["diabetes"]==0]
X_train_nonDM.to_csv("X_train_nonDM.csv")

X_test_DM=X_test[X_test["diabetes"]==1]
X_test_nonDM=X_test[X_test["diabetes"]==0]
X_test_DM.to_csv("X_test_DM.csv")
X_test_nonDM.to_csv("X_test_nonDM.csv")